🌲 Interactive Manual Placement Guide (v26+)

With the latest updates, the editor is more stable and provides better feedback for collision detection. This tool allows you to fine-tune your clusters and squeeze out those final decimals in your score!
🛠 How to Use the Editor

    Select & Move:

        Find the red dots at the center of each tree.

        Click and drag any red dot to reposition a tree. The tree polygon will follow your movement in real-time.

    Rotation:

        Click on a tree (or its center dot) to select it. The dot will turn yellow.

        Use the "Rotation" slider above the plot to rotate the selected tree from -180° to 180°.

    Collision Detection (Visual Feedback):

        Green: The tree is safely placed with no overlaps.

        Black: The tree is overlapping another tree. Even if they look close, the mathematical check requires a tiny gap (eps=0.001). Adjust the position or rotation until the trees turn green again.

    Real-Time Scoring:

        Watch the "Score" text above the plot. It updates instantly as you move objects.

        The blue dashed box represents the current Bounding Box (s×s). Your goal is to make this box as small as possible while keeping all trees green.

💾 Critical: Saving Your Progress

To ensure your manual adjustments are written to submission.csv, follow these steps:

    Check the Receiver: Ensure the Textarea box above the button is visible.

    The "Two-Click" Rule:

        Click "SAVE CSV" once. This triggers the JavaScript to grab the coordinates from the map.

        If you see the message ⚠️ Click again to confirm..., click the button a second time.

    Verification: Look for the ✅ SAVED TO submission.csv message. This confirms the Python backend has successfully updated your file.

📝 Technical Notes

    Targeting Groups: You can change the TARGET_GROUP variable in the code to edit different clusters.

    Input Initialization: Use the separate Copy Script to bring /kaggle/input/why-not/submission.csv into your working directory before you start editing.

    Precision: The tool saves coordinates with 12-decimal precision (e.g., x1.234567890123), ensuring no data is lost during the manual override.

In [ ]:
import shutil
import os

# Define file paths
SUBMISSION_PATH_IN = "/kaggle/input/santa-submission/submission.csv"
SUBMISSION_PATH_OUT = "submission.csv"

# Execute file copy
try:
    shutil.copy(SUBMISSION_PATH_IN, SUBMISSION_PATH_OUT)
    print(f"✅ Successfully copied from {SUBMISSION_PATH_IN} to {SUBMISSION_PATH_OUT}")
    print(f"File size: {os.path.getsize(SUBMISSION_PATH_OUT)} bytes")
except FileNotFoundError:
    print(f"❌ Error: The source file was not found at {SUBMISSION_PATH_IN}")
except Exception as e:
    print(f"❌ Error during copying: {e}")

In [ ]:
import pandas as pd
import numpy as np
from bokeh.plotting import figure, output_notebook, show
from bokeh.models import ColumnDataSource, PointDrawTool, HoverTool, CustomJS, Slider, Div
from bokeh.layouts import column
import ipywidgets as widgets
from IPython.display import display, clear_output, Javascript
import json

output_notebook()

# --- 1. DATA ---
TARGET_GROUP = 99

TREE_PTS = np.array([
    (0,0.8),(0.125,0.5),(0.0625,0.5),(0.2,0.25),(0.1,0.25),(0.35,0),
    (0.075,0),(0.075,-0.2),(-0.075,-0.2),(-0.075,0),(-0.35,0),
    (-0.1,0.25),(-0.2,0.25),(-0.0625,0.5),(-0.125,0.5)
])

try:
    df = pd.read_csv(SUBMISSION_PATH_OUT)
except:
    df = pd.read_csv("submission.csv")

mask = df.id.str.startswith(f"{TARGET_GROUP:03d}_")
gdf = df[mask].copy()

xs = gdf.x.str[1:].astype(float).values
ys = gdf.y.str[1:].astype(float).values
ds = gdf.deg.str[1:].astype(float).values
ids = gdf.id.values

def get_polys(cx, cy, degs):
    px, py = [], []
    for x, y, d in zip(cx, cy, degs):
        r = np.radians(d)
        px.append(list(TREE_PTS[:, 0] * np.cos(r) - TREE_PTS[:, 1] * np.sin(r) + x))
        py.append(list(TREE_PTS[:, 0] * np.sin(r) + TREE_PTS[:, 1] * np.cos(r) + y))
    return px, py

def calc_score(x, y, deg):
    minX = minY = float('inf')
    maxX = maxY = float('-inf')
    for xi, yi, di in zip(x, y, deg):
        r = np.radians(di)
        cos_r, sin_r = np.cos(r), np.sin(r)
        for pt in TREE_PTS:
            cx = pt[0] * cos_r - pt[1] * sin_r + xi
            cy = pt[0] * sin_r + pt[1] * cos_r + yi
            minX = min(minX, cx); maxX = max(maxX, cx)
            minY = min(minY, cy); maxY = max(maxY, cy)
    side = max(maxX - minX, maxY - minY)
    return side * side, (minX + maxX) / 2, (minY + maxY) / 2, side

initial_score, bx, by, bs = calc_score(xs, ys, ds)
px, py = get_polys(xs, ys, ds)

source = ColumnDataSource(data=dict(
    x=list(xs), y=list(ys), px=px, py=py, 
    name=list(ids), deg=list(ds),
    color=['green'] * len(xs)
))
bbox_source = ColumnDataSource(data=dict(x=[bx], y=[by], s=[bs]))

# --- 2. VISUALIZATION ---
p = figure(x_range=(min(xs)-3, max(xs)+3), y_range=(min(ys)-3, max(ys)+3),
           width=700, height=700, title=f"Group {TARGET_GROUP}")

p.rect(x='x', y='y', width='s', height='s', source=bbox_source, 
       fill_alpha=0.05, fill_color="blue", line_color="blue", line_dash="dashed")

p.patches('px', 'py', source=source, color='color', alpha=0.5, line_color="black")
centers = p.scatter('x', 'y', source=source, size=15, color="red", alpha=0.8, selection_color="yellow")

score_div = Div(text=f"<div style='color:blue; font-size:18px;'><b>Initial Score:</b> {initial_score:.10f}</div>")
rot_slider = Slider(start=-180, end=180, value=0, step=1, title="Rotation")

callback = CustomJS(args=dict(source=source, bbox=bbox_source, pts=TREE_PTS.tolist(), 
                               slider=rot_slider, div=score_div), code="""
    const d = source.data;
    const x = d['x'], y = d['y'], deg = d['deg'];
    const px = d['px'], py = d['py'];
    const colors = d['color'];
    const sel = source.selected.indices;
    
    if (cb_obj === slider && sel.length > 0) { deg[sel[0]] = slider.value; }
    if (cb_obj === source && sel.length > 0) { slider.value = deg[sel[0]]; }
    
    function segmentsIntersect(a1, a2, b1, b2) {
        const det = (a2.x - a1.x) * (b2.y - b1.y) - (a2.y - a1.y) * (b2.x - b1.x);
        if (Math.abs(det) < 0.000001) return false; // Nearly parallel
        const l = ((b2.y - b1.y) * (b2.x - a1.x) + (b1.x - b2.x) * (b2.y - a1.y)) / det;
        const g = ((a1.y - a2.y) * (b2.x - a1.x) + (a2.x - a1.x) * (b2.y - a1.y)) / det;
        // Add epsilon to ignore micro-touches at edges
        const eps = 0.001; 
        return (l > eps && l < 1-eps) && (g > eps && g < 1-eps);
    }

    let minX = Infinity, maxX = -Infinity, minY = Infinity, maxY = -Infinity;
    colors.fill('green');

    for (let i = 0; i < x.length; i++) {
        const r = deg[i] * Math.PI / 180;
        const cos = Math.cos(r), sin = Math.sin(r);
        for (let j = 0; j < pts.length; j++) {
            const cx = pts[j][0] * cos - pts[j][1] * sin + x[i];
            const cy = pts[j][0] * sin + pts[j][1] * cos + y[i];
            px[i][j] = cx; py[i][j] = cy;
            minX = Math.min(minX, cx); maxX = Math.max(maxX, cx);
            minY = Math.min(minY, cy); maxY = Math.max(maxY, cy);
        }
    }

    for (let i = 0; i < x.length; i++) {
        for (let j = i + 1; j < x.length; j++) {
            // Check only if trees are very close
            if (Math.pow(x[i]-x[j], 2) + Math.pow(y[i]-y[j], 2) < 0.8) {
                let hit = false;
                for (let k = 0; k < pts.length; k++) {
                    const s1a = {x: px[i][k], y: py[i][k]}, s1b = {x: px[i][(k+1)%pts.length], y: py[i][(k+1)%pts.length]};
                    for (let m = 0; m < pts.length; m++) {
                        const s2a = {x: px[j][m], y: py[j][m]}, s2b = {x: px[j][(m+1)%pts.length], y: py[j][(m+1)%pts.length]};
                        if (segmentsIntersect(s1a, s1b, s2a, s2b)) { hit = true; break; }
                    }
                    if (hit) break;
                }
                if (hit) { colors[i] = 'black'; colors[j] = 'black'; }
            }
        }
    }

    const side = Math.max(maxX - minX, maxY - minY);
    bbox.data['x'] = [(minX + maxX) / 2]; bbox.data['y'] = [(minY + maxY) / 2]; bbox.data['s'] = [side];
    div.text = "<div style='color:red; font-size:20px;'><b>Score:</b> " + (side*side).toFixed(10) + "</div>";
    
    window.treeData = {x: Array.from(x), y: Array.from(y), deg: Array.from(deg)};
    bbox.change.emit(); source.change.emit();
""")

# Setup triggers and tools
source.js_on_change('data', callback)
source.js_on_change('selected', callback)
rot_slider.js_on_change('value', callback)
draw_tool = PointDrawTool(renderers=[centers], add=False)
p.add_tools(draw_tool, HoverTool(tooltips=[("ID", "@name"), ("Deg", "@deg{0.0}")]))
p.toolbar.active_tap = draw_tool
show(column(score_div, rot_slider, p))

# --- 3. SAVING ---
save_output = widgets.Output()
# Add class 'save-receiver' for precise JS targeting
data_receiver = widgets.Textarea(value='', layout={'width': '100%', 'height': '50px'}).add_class("save-receiver")

def on_save_clicked(b):
    with save_output:
        clear_output()
        display(Javascript("""
            const data = window.treeData || null;
            if (data) {
                const ta = document.querySelector('.save-receiver textarea');
                if (ta) {
                    ta.value = JSON.stringify(data);
                    ta.dispatchEvent(new Event('input', { bubbles: true }));
                }
            }
        """))
        import time
        time.sleep(0.6) # Allow time for JS to Python transfer
        try:
            data_str = data_receiver.value
            if data_str:
                data = json.loads(data_str)
                df_new = df.copy()
                df_new.loc[mask, 'x'] = [f"x{v:.12f}" for v in data['x']]
                df_new.loc[mask, 'y'] = [f"y{v:.12f}" for v in data['y']]
                df_new.loc[mask, 'deg'] = [f"d{v:.12f}" for v in data['deg']]
                df_new.to_csv(SUBMISSION_PATH_OUT, index=False)
                print(f"✅ SAVED")
                data_receiver.value = ''
            else:
                print("⚠️ Click again to confirm data transfer...")
        except Exception as e:
            print(f"❌ ERROR: {e}")

btn_save = widgets.Button(description="💾 SAVE CSV", button_style='success', layout={'width': '300px', 'height': '50px'})
btn_save.on_click(on_save_clicked)
display(widgets.VBox([data_receiver, btn_save, save_output]))

This notebook uses Simulated Annealing (SA) to further optimize tree placements and reduce the total score. 💡 How it works

We take an existing high-quality solution and try to "squeeze" it even further:

Perturb: We run the bbox3 solver with random parameters to find a new configuration for the trees.

Evaluate: If the new score is better, we keep it.

Escape: If the score is slightly worse, we might still accept it (based on "temperature"). This helps the algorithm avoid getting stuck in a local optimum.

🤝 Acknowledgments

This notebook is based on the great work of the Kaggle community:

- https://www.kaggle.com/code/daniilkrizhanovskyi/new-year-same-old-bbox;
 
- https://www.kaggle.com/code/datafad/a-bit-better;

- https://www.kaggle.com/code/chistyakov/squeezing-bbox-solutio;

- https://www.kaggle.com/code/datafad/new-year-same-old-bbox;

- https://www.kaggle.com/code/saspav/santa-submission

- https://www.kaggle.com/code/yongsukprasertsuk/santa2025-bbox-optimization

- https://www.kaggle.com/code/chistyakov/squeezing-bbox-solution

- https://www.kaggle.com/code/datafad/single-group-optimizer-by-manoj

- https://www.kaggle.com/code/jazivxt/why-not

- /kaggle/input/k/datafad/new-year-same-old-bbox/submission.csv



In [ ]:
# ==============================
# SIMULATED ANNEALING – VARIANT A
# (FIXED INPUT PATHS)
# ==============================

import os, shutil, subprocess, random, math, tempfile
import pandas as pd, numpy as np
from decimal import Decimal, getcontext
from shapely.geometry import Polygon
from shapely import affinity
from shapely.strtree import STRtree

# ------------------------------
# CONFIG (FIXED AS REQUESTED)
# ------------------------------
START_SUB = SUBMISSION_PATH_IN # Choose SUBMISSION_PATH_IN for original or SUBMISSION_PATH_OUT to manual
START_BBOX = "/kaggle/input/k/datafad/new-year-same-old-bbox/bbox3"

WORK_SUB = "submission.csv"
BBOX = "./bbox3"

MAX_ITERS = 10
T0 = 1e-3
T_MIN = 1e-5
ALPHA = 0.99

getcontext().prec = 25
SCALE = Decimal("1e18")

# ------------------------------
# INITIAL SETUP (MINIMAL FIX)
# ------------------------------
if not os.path.exists(WORK_SUB):
    shutil.copy(START_SUB, WORK_SUB)
    print("✅ Initial submission copied")

if not os.path.exists(BBOX):
    shutil.copy(START_BBOX, BBOX)
    os.chmod(BBOX, 0o755)
    print("✅ bbox3 copied & permissions set")

# ------------------------------
# GEOMETRY & SCORING (UNCHANGED)
# ------------------------------
class ChristmasTree:
    def __init__(self,x,y,d):
        self.x=Decimal(x); self.y=Decimal(y); self.d=Decimal(d)
        tw,th=Decimal("0.15"),Decimal("0.2")
        bw,mw,ow=Decimal("0.7"),Decimal("0.4"),Decimal("0.25")
        tip,t1,t2=Decimal("0.8"),Decimal("0.5"),Decimal("0.25")
        base,tbot=Decimal("0.0"),-th
        pts=[(0,tip),(ow/2,t1),(ow/4,t1),(mw/2,t2),(mw/4,t2),
             (bw/2,base),(tw/2,base),(tw/2,tbot),
             (-tw/2,tbot),(-tw/2,base),(-bw/2,base),
             (-mw/4,t2),(-mw/2,t2),(-ow/4,t1),(-ow/2,t1)]
        p=Polygon([(float(px*SCALE),float(py*SCALE)) for px,py in pts])
        p=affinity.rotate(p,float(self.d),origin=(0,0))
        self.p=affinity.translate(p,float(self.x*SCALE),float(self.y*SCALE))

def load_group(df,n):
    g=df[df.id.str.startswith(f"{n:03d}_")]
    return [ChristmasTree(r.x[1:],r.y[1:],r.deg[1:]) for _,r in g.iterrows()]

def has_overlap(trees):
    if len(trees)<2: return False
    polys=[t.p for t in trees]
    idx=STRtree(polys)
    for i,p in enumerate(polys):
        for j in idx.query(p):
            if i!=j and p.intersects(polys[j]) and not p.touches(polys[j]):
                return True
    return False

def eval_df(df):
    tot=0.0
    for n in range(1,201):
        t=load_group(df,n)
        pts=np.concatenate([np.asarray(x.p.exterior.xy).T/1e18 for x in t])
        mn,mx=pts.min(0),pts.max(0)
        s=max(mx[0]-mn[0],mx[1]-mn[1])
        tot+=(s*s)/n
        if has_overlap(t):
            return 1e18
    return tot

def eval_submission(path):
    return eval_df(pd.read_csv(path))

# ------------------------------
# SA STEP
# ------------------------------
def propose_and_score():
    n = random.randint(50, 1000)
    r = random.randint(10, 100)

    with tempfile.TemporaryDirectory() as tmp:
        shutil.copy(WORK_SUB, f"{tmp}/submission.csv")
        shutil.copy(BBOX, f"{tmp}/bbox3")
        os.chmod(f"{tmp}/bbox3", 0o755)

        cwd = os.getcwd()
        os.chdir(tmp)

        subprocess.run(
            ["./bbox3", "-n", str(n), "-r", str(r)],
            stdout=subprocess.DEVNULL,
            stderr=subprocess.DEVNULL
        )

        score = eval_submission("submission.csv")

        # ⬇️ КЛЮЧОВИЙ ФІКС: копіюємо у стабільний файл
        stable_tmp = tempfile.NamedTemporaryFile(delete=False, suffix=".csv")
        shutil.copy("submission.csv", stable_tmp.name)

        os.chdir(cwd)

    return score, stable_tmp.name
# ------------------------------
# SIMULATED ANNEALING LOOP
# ------------------------------
current_score = eval_submission(WORK_SUB)
best_score = current_score
T = T0

print("🔥 Initial score:", current_score)

for it in range(1, MAX_ITERS+1):
    new_score, new_sub = propose_and_score()
    delta = new_score - current_score

    accept = delta < 0 or random.random() < math.exp(-delta / T)

    if accept:
        shutil.copy(new_sub, WORK_SUB)
        current_score = new_score

        if new_score < best_score:
            best_score = new_score
            print(f"🏆 NEW BEST @ iter {it}: {best_score:.6f}")

    T = max(T * ALPHA, T_MIN)

    if it % 20 == 0:
        print(f"Iter {it} | T={T:.2e} | current={current_score:.6f} | best={best_score:.6f}")

print("\n✅ FINAL BEST SCORE:", best_score)
print("📄 submission.csv READY for Kaggle submit")
